In [1]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import os.path as path
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from faker import Factory
from scipy import interpolate
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math 
import itertools 
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from ipywidgets import HBox, VBox
from ipywidgets import IntSlider, Output
from IPython.display import clear_output
import matplotlib.colors as mcolors
import h5py
import re
import matplotlib
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables, calibration, ion_selection
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.mc import mc_tools
from pyccapt.calibration.leap_tools import ccapt_tools
from pyccapt.calibration.calibration_tools import mc_plot, gui_ion_select

In [2]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [3]:
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()
display(tdc, flightPathLength_d)

Dropdown(description='Data mode:', options=('PyCCAPT', 'leap_epos', 'leap_pos', 'ato_v6'), value='PyCCAPT')

FloatText(value=110.0, description='Flight path length:')

$$\textbf{You can specify which dataset to use in below block}$$

In [4]:
# exctract needed data from Pandas data frame as an numpy array
# create an instance of the Variables opject
variables = share_variables.Variables()
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
dataset_main_path = os.path.dirname(dataset_main_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path + '/ions_selection/'
variables.result_data_name = 'range_' + variables.dataset_name
variables.result_path = dataset_main_path + '/' + '/ions_selection/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
    
# Create data farame out of hdf5 file dataset
data = data_tools.load_data(dataset_path, tdc.value, mode='processed')
# extract data from the path and create the Variable object
data_tools.extract_data(data, variables, flightPathLength_d.value, max_mc.value)

NameError: name 'dataset_path' is not defined

In [5]:
print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')
print('Total number of Ions:', len(data))
data

The data will be saved on the path: 
The dataset name after saving is: 
The figures will be saved on the path: 


NameError: name 'data' is not defined

In [6]:
interact_manual(mc_plot.hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), log=widgets.Dropdown(options=[('True', True), ('Flase', False)]),
                target=widgets.Dropdown(options=[('mc_c', 'mc_c'), ('tof_c', 'tof_c')]), mode=widgets.Dropdown(options=[('normal', 'normal'), ('normalized', 'normalized')]),
                prominence=widgets.IntText(value=100), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=variables.max_tof), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=widgets.Dropdown(options=[('True', True), ('Flase', False)]), range_plot=fixed(False), selected_area=fixed(False), print_info=fixed(True));

interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='log', options=(('Tru…

In [7]:
bin_size=widgets.FloatText(value=0.1, description='bin size:')
prominence=widgets.IntText(value=50, description='peak prominance:')
distance=widgets.IntText(value=1, description='peak distance:')
lim_tof=widgets.IntText(value=400, description='lim tof/mc:')
percent=widgets.IntText(value=50, description='percent MRP:')
index_fig = widgets.IntText(value=1, description='fig index:')
plot_peak = widgets.Dropdown(
    options=[('True', True), ('False', False)],
    description='plot peak:'
)

def hist_plot_p(variables, plot):
    
    with out:
        clear_output(True)
        # clear the peak_idx
        variables.peaks_idx = []
        mc_plot.hist_plot(variables, bin_size.value, log=True, target='mc_c', mode='normal', prominence=prominence.value, distance=distance.value, percent=percent.value,
                          selector='peak', figname=index_fig.value, lim=lim_tof.value, peaks_find_plot=plot_peak.value, print_info=False)

def hist_plot_r(variables, plot):
    
    with out:
        clear_output(True)
        print('=============================')
        print('Press left click to draw a line')
        print('Press right click to remove a line')
        print('Press r to remove all the line')
        print('Hold shift and use mouse scroll for zooming on x axis')
        print('Hold ctrl and left mouse bottom to move a line')
        print('=============================')
        mc_plot.hist_plot(variables, bin_size.value, log=True, target='mc_c', mode='normal', prominence=prominence.value, distance=distance.value, percent=percent.value,
                          selector='range', figname=index_fig.value, lim=lim_tof.value, peaks_find_plot=True, ranging_mode=True, save_fig=False, print_info=False)


In [8]:
# element calculate
peak_val = widgets.FloatText(value=1.1, description='peak value:')
charge = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)
aboundance_threshold = widgets.FloatText(value=0.0, description='threshold aboundance:', min=0, max=1, step=0.1)
num_element = widgets.IntText(value=5, description='num element:')
# formula calculate
formula_m = widgets.Text(
    value='{12}C1{16}O2',
    placeholder='Type a formula  {12}C1{16}O2',
    description='Isotope formula:',
    disabled=False
)

molecule_charge = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)

aboundance_threshold_molecule = widgets.FloatText(value=0.0, description='threshold aboundance:', min=0, max=1, step=0.1)

# molecule create
formula_com = widgets.Text(
    value='',
    placeholder="H, O",
    description='Elements:',
    disabled=False
)
complexity = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='complexity:'
)

charge_com = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)

In [16]:
plot_button_p = widgets.Button(
    description='plot hist',
)

plot_button_r = widgets.Button(
    description='plot hist',
)

plot_button = widgets.Button(
    description='plot hist',
)


find_elem_button = widgets.Button(
    description='find element',
)

formula_button = widgets.Button(
    description='manual formula',
)

add_ion_button = widgets.Button(
    description='add ion',
)
romove_ion_button = widgets.Button(
    description='remove ion',
)
show_color = widgets.Button(
    description='show color',
)
change_color = widgets.Button(
    description='change color',
)

color_picker = widgets.ColorPicker(description='Select a color:')
row_index = widgets.IntText(value=0, description='index row:')
mass_unknown = widgets.FloatText(value=0.0, description='unknown mass:')
@plot_button_p.on_click
def plot_on_click_p(b, plot=True):
    hist_plot_p(variables, plot)
    
@plot_button_r.on_click
def plot_on_click_r(b, plot=True):
    hist_plot_r(variables, plot)
    
    
@find_elem_button.on_click
def vol_on_click(b,):
    with output2:
        clear_output(True)
        df1 = ion_selection.load_elements(formula_com.value, aboundance_threshold.value, charge.value, variables=variables)
        df2 = ion_selection.molecule_create(formula_com.value, complexity.value, charge.value, aboundance_threshold.value, variables)
        df = pd.concat([df1, df2], axis=0)
        df = df.iloc[(df['mass'] - peak_val.value).abs().argsort()]
        # Reset the index to maintain a clean index order
        df.reset_index(drop=True, inplace=True)
        # Save to a pickle file
        import pickle
        with open('data.pkl', 'wb') as file:
            pickle.dump(df, file)
        # gui_ion_select.open_gui(df, variables)
        display(df)
    
@formula_button.on_click
def manual_formula(b,):
    with output2:
        if formula_m.value == '':
            print("Input is empty. Type the formula.")
        else:
            df = ion_selection.molecule_manual(formula_m.value, molecule_charge.value, latex=True, variables=variables)
            clear_output(True)
            display(df)

@add_ion_button.on_click
def add_ion_to_range_dataset(b,):
    ion_selection.ranging_dataset_create(variables, row_index.value, mass_unknown.value)
    with output3:
        clear_output(True)
        display(variables.range_data)
@romove_ion_button.on_click
def remove_ion_to_range_dataset(b,):
    if len(variables.range_data) >= 1:
        variables.range_data = variables.range_data.drop(len(variables.range_data) - 1)
        with output3:
            clear_output(True)
            display(variables.range_data)
        
@show_color.on_click
def show_color_ions(b,):
    with output3:
        clear_output(True)
        display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))
@change_color.on_click
def change_color_m(b,):
    with output3:
        selected_color = mcolors.to_hex(color_picker.value)
        variables.range_data.at[row_index.value, 'color'] = selected_color
        clear_output(True)
        display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

        # Create "Next" and "Previous" buttons
start_button = widgets.Button(description="Start")
next_button = widgets.Button(description="Next")
prev_button = widgets.Button(description="Previous")
reset_h_line_button = widgets.Button(description="Reset")
all_peaks_button = widgets.Button(description="Add all peaks")


# Define button click events
@start_button.on_click
def start_peak(_):
    variables.h_line_pos = []
    print('=============================')
    print('Press left click to draw a line')
    print('Press right click to remove a line')
    print('Press r to remove all the line')
    print('Press a to automatically draw lines')
    print('Hold shift and use mouse scroll for zooming on x axis')
    print('Hold ctrl and left mouse bottom to move a line')
    print('=============================')
    variables.peaks_index = 0
    peak_val.value = variables.peaks_x_selected[variables.peaks_index]
    peaks_x_selected = variables.peaks_x_selected[:]
    variables.peaks_x_selected = [variables.peaks_x_selected[variables.peaks_index]]
    print('peak idc:', variables.peaks_index, 'Peak location:', peak_val.value )
    mask = (variables.mc_c < peak_val.value  + 5) & (variables.mc_c > peak_val.value  - 5)
    mc = np.copy(variables.mc_calib)
    peaks_x = np.copy(variables.peak_x)
    variables.mc_c = variables.mc_c[mask]
    hist_plot_r(variables, plot=True)
    variables.mc_c = mc
    variables.peak_x = peaks_x
    variables.peaks_x_selected = peaks_x_selected[:]
@next_button.on_click
def next_peak(_):
    variables.h_line_pos = []
    variables.peaks_index = variables.peaks_index + 1
    peak_val.value = variables.peaks_x_selected[variables.peaks_index]
    peaks_x_selected = variables.peaks_x_selected[:]
    variables.peaks_x_selected = [variables.peaks_x_selected[variables.peaks_index]]
    print('peak idc:', variables.peaks_index, 'Peak location:', peak_val.value )
    mask = (variables.mc_c < peak_val.value  + 5) & (variables.mc_c > peak_val.value  - 5)
    mc = np.copy(variables.mc_calib)
    peaks_x = np.copy(variables.peak_x)
    variables.mc_c = variables.mc_c[mask]
    hist_plot_r(variables, plot=True)
    variables.mc_c = mc
    variables.peak_x = peaks_x
    variables.peaks_x_selected = peaks_x_selected[:]
@prev_button.on_click
def prev_peak(_):
    variables.h_line_pos = []
    variables.peaks_index = variables.peaks_index - 1
    peak_val.value = variables.peaks_x_selected[variables.peaks_index]
    peaks_x_selected = variables.peaks_x_selected[:]
    variables.peaks_x_selected = [variables.peaks_x_selected[variables.peaks_index]]
    print('peak idc:', variables.peaks_index, 'Peak location:', peak_val.value )
    mask = (variables.mc_c < peak_val.value  + 5) & (variables.mc_c > peak_val.value  - 5)
    mc = np.copy(variables.mc_calib)
    peaks_x = np.copy(variables.peak_x)
    variables.mc_c = variables.mc_c[mask] 
    hist_plot_r(variables, plot=True)
    variables.mc_c = mc
    variables.peak_x = peaks_x
    variables.peaks_x_selected = peaks_x_selected[:]

@reset_h_line_button.on_click
def rest_h_line(_):
    variables.h_line_pos = []
@all_peaks_button.on_click
def select_all_peaks(_):
    variables.peaks_idx = variables.peak_x
    
tab1 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, plot_peak, HBox(children=[plot_button_p, all_peaks_button])])
tab2 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, HBox(children=[VBox(children=[plot_button_r, start_button, next_button, prev_button, reset_h_line_button])])])
tab4 = VBox(children=[HBox(children=[VBox(children=[peak_val, charge, aboundance_threshold, formula_com, complexity, find_elem_button]),
                                    VBox(children=[row_index, color_picker, mass_unknown, add_ion_button, romove_ion_button, show_color, change_color]), 
                                    VBox(children=[formula_m, molecule_charge, formula_button])])])

tabs1 = widgets.Tab(children=[tab1, tab2])
tabs2 = widgets.Tab(children=[tab4])
tabs1.set_title(0, 'peak finder')
tabs1.set_title(1, 'rangging')
tabs2.set_title(0, 'element finder')
# Create two Output widgets to capture the output of each plot
out = Output()
output2 = Output()
output3 = Output()

# Create an HBox to display the buttons side by side
buttons_layout = widgets.HBox([tabs1, tabs2])

# Create a VBox to display the output widgets below the buttons
output_layout_h = widgets.HBox([out, output3])
output_layout = widgets.VBox([output_layout_h, output2])


# Display the buttons and the output widgets
display(buttons_layout, output_layout)


with output3:
    display(variables.range_data)

In [10]:
interact_manual(mc_plot.hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), log=widgets.Dropdown(options=[('True', True), ('Flase', False)]),
                target=widgets.Dropdown(options=[('mc_c', 'mc_c')]), mode=widgets.Dropdown(options=[('normal', 'normal'), ('normalized', 'normalized')]),
                prominence=widgets.IntText(value=100), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=widgets.Dropdown(options=[('True', True), ('Flase', False)]), range_plot=widgets.Dropdown(options=[('True', True), ('Flase', False)]), selected_area=fixed(False), print_info=fixed(True));

interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='log', options=(('Tru…

In [20]:
display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,${}^{1}H^{+}$,1.010000,1.067455,0.924459,1.216181,#b2aa2d,H,1,1,1
1,${}^{27}Al^{2+}$,13.490000,14.478278,13.812979,15.404862,#e7e0d1,Al,1,27,2
2,${}^{27}Al^{+}$,26.980000,29.290230,27.923454,31.444492,#e7e0d1,Al,1,27,1


In [21]:
variables.range_data.dtypes

ion         object
mass       float64
mc         float64
mc_low     float64
mc_up      float64
color       object
element     object
complex      int32
isotope      int64
charge       int32
dtype: object

In [22]:
# save the new data
name_save_file = variables.result_data_path + '/' + variables.result_data_name + '.h5'
data_tools.store_df_to_hdf(variables.range_data,  'df', name_save_file)

In [23]:
# save data in csv format
name_save_file = variables.result_data_path + '/' + variables.result_data_name + '.csv'
data_tools.store_df_to_csv(variables.range_data, name_save_file)